In [ ]:
import numpy as np 
import pandas as pd

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(f"Directory: {dirname}")
    for filename in filenames:
        print(f"  - {filename}")


In [ ]:
import pandas as pd


data_path = '/kaggle/input/MIT-BIH.csv'

# قراءة البيانات
data = pd.read_csv(data_path)
data.head()


In [ ]:
data.shape

In [ ]:
data.drop('Unnamed: 0', axis=1, inplace=True)

In [ ]:
pd.options.display.float_format='{:,.3f}'.format
data.describe()

In [ ]:
data.duplicated().sum()

In [ ]:
data["target"].value_counts()

***The heartbeats are categorized into
89694 for Normal beats, 6487 for Ventricular ectopic beats,
2814 for Supraventricular ectopic beats, 779 for Fusion beats,
and 24 for Unknown beats.--> "Imbalanced Data"***


In [ ]:
from sklearn.model_selection import train_test_split
from collections import Counter

x= data.drop('target',axis=1)
y= data['target'] 

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.25, stratify=y, random_state=42
)


print("Distribution of classes in training data:", Counter(y_train))
print("Distribution of classes in testing data:", Counter(y_test))


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

rus = RandomUnderSampler(sampling_strategy={1: 25000}, random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

ros = RandomOverSampler(sampling_strategy={3: 25000, 4: 25000, 0: 25000,2:25000}, random_state=42)
X_train_balanced, y_train_balanced = ros.fit_resample(X_train_resampled, y_train_resampled)

print("Final distribution of labeled data:", Counter(y_train_balanced))


In [ ]:
pip install imblearn

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

rus = RandomUnderSampler(sampling_strategy={1: 25000}, random_state=42)
X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)

ros = RandomOverSampler(sampling_strategy={3: 25000, 4: 25000, 0: 25000,2:25000}, random_state=42)
X_train_balanced, y_train_balanced = ros.fit_resample(X_train_resampled, y_train_resampled)

print("Final distribution of labeled data:", Counter(y_train_balanced))


In [ ]:
import pandas as pd

balanced_data = pd.DataFrame(X_train_balanced)
balanced_data['target'] = y_train_balanced
print(balanced_data['target'].value_counts())

# LSTM 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
model = Sequential([
    LSTM(128, input_shape=(125000, 125000), return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(5, activation='softmax')  
])


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
X_train_balanced_np = X_train_balanced.to_numpy()  
X_train_balanced_reshaped = X_train_balanced_np.reshape(X_train_balanced_np.shape[0], X_train_balanced_np.shape[1], 1)


history = model.fit(
    X_train_balanced_reshaped, 
    y_train_balanced,          
    epochs=10,
    batch_size=64,
    validation_split=0.2
)


In [ ]:

X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)


y_pred_prob = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_prob, axis=1)
print("Classification Report_LSTM:")
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=[f"Class {i}" for i in range(5)],
            yticklabels=[f"Class {i}" for i in range(5)])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# GRU 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential()

model.add(GRU(64, input_shape=(X_train_balanced.shape[1], 1), return_sequences=False))

model.add(Dropout(0.2))

model.add(Dense(5, activation='softmax')) 

optimizer = Adam(learning_rate=0.001, decay=1e-6)

model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()


history = model.fit(
    X_train_balanced, y_train_balanced,
    epochs=25,
    batch_size=512,
    validation_split=0.2
)


In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

In [ ]:
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_pred_prob = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_prob, axis=1)
print("Classification Report_GRU:")
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=[f"Class {i}" for i in range(5)],
            yticklabels=[f"Class {i}" for i in range(5)])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# Hybrid Model LSTM and GRU 

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dropout, Dense, Flatten
from tensorflow.keras.optimizers import Adam

model = Sequential()

# LSTM Layer 
model.add(LSTM(units=64, input_shape=(180, 1), return_sequences=True, activation='tanh'))

# GRU Layer
model.add(GRU(units=64, return_sequences=False, activation='tanh'))

# Dropout Layer
model.add(Dropout(0.2))

# Output Layer
model.add(Dense(5, activation='softmax'))  

# Compile the model
optimizer = Adam(learning_rate=0.001, decay=1e-6)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()


In [ ]:
History = model.fit(
    X_train_balanced_reshaped,  
    y_train_balanced,           
    batch_size=512,
    epochs=10,
    validation_split=0.2
)

In [ ]:
X_test_reshaped = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_pred_prob = model.predict(X_test_reshaped)
y_pred = np.argmax(y_pred_prob, axis=1)
print("Classification Report_LSTM_GRU:")
print(classification_report(y_test, y_pred))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=[f"Class {i}" for i in range(5)],
            yticklabels=[f"Class {i}" for i in range(5)])
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()

# "The hybrid system consisting of LSTM and GRU achieved the highest accuracy of 96.33%."